In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
sys.path.append('../experiments')
import dvu
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import neuro.features.qa_questions as qa_questions
import joblib
from tqdm import tqdm
from neuro import analyze_helper, viz
fit_encoding = __import__('02_fit_encoding')
dvu.set_style()

# results_dir = analyze_helper.best_results_dir
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/feb13_2025_test_tabpfn'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
# joblib.dump({'r': rr, 'cols_varied': cols_varied, 'mets': mets}, 'results.pkl')
# data = joblib.load('results.pkl')
# rr, cols_varied, mets = data['r'], data['cols_varied'], data['mets']
metric_sort = 'corrs_tune_pc_weighted_mean'

In [ ]:
r.loc[
    (r.num_stories == num_stories) & (r.encoding_model == encoding_model), 'corrs_test_mean'].mean()

In [ ]:
for ndel in rr.ndelays.unique():
    r = rr[rr.ndelays == ndel]
    r = r[r.subject.isin(['S01', 'S02', 'S03'])]

    # add AVG over subjects
    for num_stories in r.num_stories.unique():
        for encoding_model in r.encoding_model.unique():
            new_row = deepcopy(r.iloc[0])
            new_row['subject'] = 'AVG'
            new_row['num_stories'] = num_stories
            new_row['encoding_model'] = encoding_model
            new_row['corrs_test_mean'] = r.loc[(r.num_stories == num_stories) & (
                r.encoding_model == encoding_model), 'corrs_test_mean'].mean()
            r = pd.concat([r, new_row.to_frame().T], ignore_index=True)

    r['grouping'] = r['num_stories'].astype(
        str).str.zfill(2) + ' ' + r['subject'].astype(str)
    # r = r[r.grouping.str.contains('AVG')]
    sns.barplot(data=r, x='grouping', y='corrs_test_mean',
                hue='encoding_model', order=sorted(r['grouping'].unique()),
                hue_order=['ridge', 'tabpfn', 'mlp'])

    plt.xticks(rotation=90)
    plt.xlabel('Num training stories & subject')
    plt.title('ndelay = ' + str(ndel))
    plt.show()